In [7]:
from my_utils import stardisting as sd
from my_utils import published_data_processing as pub
from my_utils import tile_processing as tp

import os
import pandas as pd
from stardist.matching import matching_dataset as scorer
from tqdm import tqdm

In [8]:
path_20x_public_downsample = "Z:\Interpolations\L - 20x - 1_"
full_dataset = pub.read_all_20x_published_data(path_20x_public_downsample)

In [9]:
path_models_sd = r"Z:\Segmentation Models\StarDist"
inputs_csv = "SD_HE_20x Candidate Scores.csv"
df = pd.read_csv(os.path.join(path_models_sd, inputs_csv))
df

,Model,Initial Model,Split Percentiles,Split State,Epochs,Learning Rate,Aug Rot90,Aug Flip,Aug Intensity,Aug Hue,Aug Blur,Tau,TP,FP,FN,Precision,Recall,F1-Score,Panoptic Quality
0,Model 01,2D_versatile_he,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3772.0,1181.0,1716.0,0.762000,0.687000,0.723000,0.528000
1,Model 02,Model 01,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3750.0,1135.0,1738.0,0.767656,0.683309,0.723031,0.528952
2,Model 03,Model 02,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3728.0,1091.0,1760.0,0.774000,0.679000,0.723000,0.531000
3,Model 04,Model 03,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3638.0,956.0,1850.0,0.792000,0.663000,0.722000,0.531000
4,Model 05,2D_versatile_he,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3695.0,1046.0,1793.0,0.779000,0.673000,0.722000,0.530000
5,Model 06,Model 05,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3662.0,1052.0,1826.0,0.777000,0.667000,0.718000,0.525000
6,Model 07,Model 06,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3726.0,1095.0,1762.0,0.773000,0.679000,0.723000,0.530000
7,Model 08,Model 07,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3698.0,1083.0,1790.0,0.773000,0.674000,0.720000,0.527000
8,Model 09,2D_versatile_he,"[75, 15, 10]",7,50,0.00001,False,False,False,False,False,0.5,3703.0,1244.0,1785.0,0.749000,0.675000,0.710000,0.513000
9,Model 10,Model 09,"[75, 15, 10]",7,50,0.00001,False,False,False,False,False,0.5,3738.0,1232.0,1750.0,0.752000,0.681000,0.715000,0.520000


In [10]:
for i, row in tqdm(df[df.isna().any(axis=1)].iterrows()):
    model_name, starting_model, splits, r_state, epochs, lr, rot90, flip, intensity, hue, blur, tau = [None] * 12
    for item, value in row.items():
        if item == 'Model':
            model_name = value
        elif item == 'Initial Model':
            starting_model = value
        elif item == 'Split Percentiles':
            splits = eval(value)
        elif item == 'Split State':
            r_state = value
        elif item == 'Epochs':
            epochs = value
        elif item == 'Learning Rate':
            lr = value
        elif item == 'Aug Rot90':
            rot90 = value
        elif item == 'Aug Flip':
            flip = value
        elif item == 'Aug Intensity':
            intensity = value
        elif item == 'Aug Hue':
            hue = value
        elif item == 'Aug Blur':
            blur = value
        elif item == 'Tau':
            tau = value
    
    if starting_model == '2D_versatile_he':
        model = sd.load_published_he_model(path_models_sd, model_name)
    elif starting_model == 'Random Initialization':
        model = sd.load_random_he_model(path_models_sd, model_name)
    else:
        model = sd.load_model(os.path.join(path_models_sd, starting_model),
                              os.path.join(path_models_sd, model_name), from_last_weights=True)
    
    trn_vld_tst = pub.split_all_data(splits, full_dataset, r_state)
    trn = {'Images': [], 'Masks': []}
    vld = {'Images': [], 'Masks': []}
    tst = {'Images': [], 'Masks': [], 'Predictions': []}
    for dataset in list(full_dataset.keys()):
        [trn['Images'].append(trn_vld_tst['Train'][dataset]['Images'][i]) for i, img in enumerate(trn_vld_tst['Train'][dataset]['Images'])]
        [trn['Masks'].append(trn_vld_tst['Train'][dataset]['Masks'][i]) for i, msk in enumerate(trn_vld_tst['Train'][dataset]['Masks'])]
        [vld['Images'].append(trn_vld_tst['Validate'][dataset]['Images'][i]) for i, img in enumerate(trn_vld_tst['Validate'][dataset]['Images'])]
        [vld['Masks'].append(trn_vld_tst['Validate'][dataset]['Masks'][i]) for i, msk in enumerate(trn_vld_tst['Validate'][dataset]['Masks'])]
        [tst['Images'].append(trn_vld_tst['Test'][dataset]['Images'][i]) for i, img in enumerate(trn_vld_tst['Test'][dataset]['Images'])]
        [tst['Masks'].append(trn_vld_tst['Test'][dataset]['Masks'][i]) for i, msk in enumerate(trn_vld_tst['Test'][dataset]['Masks'])]
    
    if rot90 or flip or intensity or hue or blur:
        aug_images, aug_masks = [], []
        for j, img in enumerate(trn['Images']):
            msk = trn['Masks'][j]
            augmenter = tp.TilePairAugmenter(img, msk, random_state=j, rotate90=rot90, flip=flip, intensity=intensity, hue=hue, blur=blur)
            aug_images.append(augmenter.image_rgb)
            aug_masks.append(augmenter.mask_gray)
        [trn['Images'].append(img) for img in aug_images]
        [trn['Masks'].append(msk) for msk in aug_masks]
    
    model = sd.configure_model_for_training(model=model, use_gpu=True, epochs=epochs, learning_rate=lr)
    model = sd.normalize_train_and_threshold(model=model, training_images=trn['Images'], training_masks=trn['Masks'],
                                             validation_images=vld['Images'], validation_masks=vld['Masks'])
    
    for img in tst['Images']:
        img = img / 255
        pred, _ = model.predict_instances(img)
        tst['Predictions'].append(pred)
    
    scores = scorer(tst['Masks'], tst['Predictions'], thresh=tau, show_progress=False)._asdict()
    df.at[i, 'TP'] = int(scores['tp'])
    df.at[i, 'FP'] = int(scores['fp'])
    df.at[i, 'FN'] = int(scores['fn'])
    df.at[i, 'Precision'] = scores['precision']
    df.at[i, 'Recall'] = scores['recall']
    df.at[i, 'F1-Score'] = scores['f1']
    df.at[i, 'Panoptic Quality'] = scores['panoptic_quality']
    

0it [00:00, ?it/s]

Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.
Using default values: prob_thresh=0.5, nms_thresh=0.4.

Ignore that, thresholds are: Thresholds(prob=0.6924782541382084, nms=0.3)
Epoch 1/200
100/100 [==============================] - 7s 57ms/step - loss: 0.5381 - prob_loss: 0.2257 - dist_loss: 1.5620 - prob_kld: 0.1012 - dist_relevant_mae: 1.5617 - dist_relevant_mse: 5.8164 - dist_dist_iou_metric: 0.5799 - val_loss: 0.4867 - val_prob_loss: 0.1674 - val_dist_loss: 1.5962 - val_prob_kld: 0.0730 - val_dist_relevant_mae: 1.5959 - val_dist_relevant_mse: 6.7749 - val_dist_dist_iou_metric: 0.5718 - lr: 3.0000e-04
Epoch 2/200
100/100 [==============================] - 4s 41ms/step - loss: 0.5078 - prob_loss: 0.2144 - dist_loss: 1.4669 - prob_kld: 0.0891 - dist_relevant_mae: 1.4665 - dist_relevant_mse: 5.2821 - dist_dist_iou_metric: 0.6058 - val_l


NMS threshold = 0.3:  75%|███████▌  | 15/20 [00:07<00:02,  1.94it/s, 0.689 -> 0.467]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:07<00:02,  1.89it/s, 0.689 -> 0.466]

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:07<00:02,  1.91it/s, 0.717 -> 0.464]


Using optimized values: prob_thresh=0.686173, nms_thresh=0.3.
Saving to 'thresholds.json'.


1it [13:31, 811.22s/it]


In [11]:
df

,Model,Initial Model,Split Percentiles,Split State,Epochs,Learning Rate,Aug Rot90,Aug Flip,Aug Intensity,Aug Hue,Aug Blur,Tau,TP,FP,FN,Precision,Recall,F1-Score,Panoptic Quality
0,Model 01,2D_versatile_he,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3772.0,1181.0,1716.0,0.762000,0.687000,0.723000,0.528000
1,Model 02,Model 01,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3750.0,1135.0,1738.0,0.767656,0.683309,0.723031,0.528952
2,Model 03,Model 02,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3728.0,1091.0,1760.0,0.774000,0.679000,0.723000,0.531000
3,Model 04,Model 03,"[75, 15, 10]",7,50,0.00010,False,False,False,False,False,0.5,3638.0,956.0,1850.0,0.792000,0.663000,0.722000,0.531000
4,Model 05,2D_versatile_he,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3695.0,1046.0,1793.0,0.779000,0.673000,0.722000,0.530000
5,Model 06,Model 05,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3662.0,1052.0,1826.0,0.777000,0.667000,0.718000,0.525000
6,Model 07,Model 06,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3726.0,1095.0,1762.0,0.773000,0.679000,0.723000,0.530000
7,Model 08,Model 07,"[75, 15, 10]",7,50,0.00005,False,False,False,False,False,0.5,3698.0,1083.0,1790.0,0.773000,0.674000,0.720000,0.527000
8,Model 09,2D_versatile_he,"[75, 15, 10]",7,50,0.00001,False,False,False,False,False,0.5,3703.0,1244.0,1785.0,0.749000,0.675000,0.710000,0.513000
9,Model 10,Model 09,"[75, 15, 10]",7,50,0.00001,False,False,False,False,False,0.5,3738.0,1232.0,1750.0,0.752000,0.681000,0.715000,0.520000


In [12]:
df.to_csv(os.path.join(path_models_sd, inputs_csv), index=False)